# 2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트

이 노트북은 LLM 기반 연구 데이터/논문 추천 시스템의 평가를 수행합니다.

**실행 환경**
- GPU: NVIDIA RTX 3080 이상
- CUDA: 11.8+
- Python: 3.10+

## 평가 (Evaluation)

In [1]:
import sys
import os
import json
import logging
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# Project root path
project_root = '/home/infidea/backup-data/paper-reco-agent'
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")

# Logging setup
os.makedirs(os.path.join(project_root, 'logs'), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(os.path.join(project_root, 'logs/evaluation.log'))
    ],
    force=True
)

print("✅ 로깅 설정 완료")

# Import evaluation modules
from src.evaluation.metrics import (
    evaluate_recommendations,
    batch_evaluate,
    format_evaluation_report
)
from src.config.settings import settings

print("✅ 평가 모듈 임포트 완료")

Project root: /home/infidea/backup-data/paper-reco-agent
Logging setup complete
Evaluation modules imported successfully


/home/infidea/backup-data/miniconda3/envs/paper-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. 환경 설정 및 평가 모듈 임포트

In [2]:
# Load test dataset
testset_path = os.path.join(project_root, 'data', 'test', 'testset.json')

with open(testset_path, 'r', encoding='utf-8') as f:
    testset = json.load(f)

print(f"테스트 데이터셋 로드 완료: {testset_path}")
print(f"테스트 카테고리 수: {len(testset.keys())}")

# Extract all test cases
all_test_cases = []
for category, cases in testset.items():
    for case in cases:
        case['category'] = category
        all_test_cases.append(case)

print(f"총 테스트 케이스: {len(all_test_cases)}개")
print(f"\n첫 번째 테스트 케이스 구조:")
print(json.dumps(all_test_cases[0], indent=2, ensure_ascii=False)[:500])

Test dataset loaded from: /home/infidea/backup-data/paper-reco-agent/data/test/testset_gemini_ver1.json
Number of test categories: 2
Total test cases: 19

First test case structure:
{
  "input_dataset": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "title": "한반도 기상 관측 데이터 세트",
    "description": "한반도 종관 기상 관측 세스템으로부터의 관측 데이터 세트",
    "keywords": [
      "한반도",
      "기상",
      "센서데이터"
    ]
  },
  "candidate_pool": {
    "papers": [
      {
        "id": "DIKO0014589705",
        "title": "쿠로시오 해류의 변동성과 동아시아 겨울 기후와의 상관성 연구",
        "abstract": "동아시아 겨울 기후를 대표하는 동아시아 겨울 몬순의 변동성은 외부적 다양한 인자들에 의해 영향을 받는다. 북서태평양에서 주요한 흐름인 쿠로시오 해류도 한반도와 주변 바다에 영향을 미치며 동아시아 기후에 영향을 주는


### 2. 테스트 데이터셋 로드

In [3]:
# Create timestamped output directory
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
eval_output_dir = os.path.join(project_root, 'figures', 'evaluation_results', timestamp)
os.makedirs(eval_output_dir, exist_ok=True)

print(f"📊 평가 결과 저장 경로: {eval_output_dir}")

# Save evaluation configuration
config_to_save = {
    'model_name': settings.MODEL_NAME,
    'embedding_model': settings.EMBEDDING_MODEL,
    'max_tokens': settings.MAX_TOKENS,
    'temperature': settings.TEMPERATURE,
    'paper_hybrid_alpha': settings.PAPER_HYBRID_ALPHA,
    'paper_hybrid_beta': settings.PAPER_HYBRID_BETA,
    'dataset_hybrid_alpha': settings.DATASET_HYBRID_ALPHA,
    'dataset_hybrid_beta': settings.DATASET_HYBRID_BETA,
    'dev_mode': settings.DEV_MODE,
    'evaluation_timestamp': timestamp,
    'testset_path': testset_path,
    'num_test_cases': len(all_test_cases)
}

config_path = os.path.join(eval_output_dir, 'config.json')
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config_to_save, f, indent=2, ensure_ascii=False)

print(f"✅ 설정 저장 완료: {config_path}")
print(f"\n설정 내용:")
print(json.dumps(config_to_save, indent=2, ensure_ascii=False))

Evaluation results will be saved to: /home/infidea/backup-data/paper-reco-agent/data/evaluation_results/20251015103509
Configuration saved to: /home/infidea/backup-data/paper-reco-agent/data/evaluation_results/20251015103509/config.json

Configuration:
{
  "model_name": "Qwen/Qwen3-14B",
  "embedding_model": "intfloat/multilingual-e5-large",
  "max_tokens": 512,
  "temperature": 0.1,
  "paper_hybrid_alpha": 0.8,
  "paper_hybrid_beta": 0.2,
  "dataset_hybrid_alpha": 0.6,
  "dataset_hybrid_beta": 0.4,
  "dev_mode": false,
  "evaluation_timestamp": "20251015103509",
  "testset_path": "/home/infidea/backup-data/paper-reco-agent/data/test/testset_gemini_ver1.json",
  "num_test_cases": 19
}


### 3. 평가 결과 저장 디렉토리 설정

In [4]:
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
import asyncio

# Initialize agent
agent = KoreanResearchRecommendationAgent()

print(f"✅ 에이전트 초기화 완료")
print(f"개발 모드: {agent.dev_mode}")
print(f"모델 정보: {agent.llm_model.get_model_info()}")

2025-10-15 10:35:21,399 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-10-15 10:35:21,402 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
2025-10-15 10:35:27,438 - src.agents.recommendation_agent - INFO - 🚀 프로덕션 모드로 실행: 실제 Qwen 모델 사용
2025-10-15 10:35:27,441 - src.models.qwen_model - INFO - 🚀 Qwen 모델 로딩 시작: Qwen/Qwen3-14B
2025-10-15 10:35:27,442 - src.models.qwen_model - INFO -    - 디바이스: cuda
2025-10-15 10:35:28,324 - src.models.qwen_model - INFO -    - FP16 모드 (~28GB VRAM)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 8/8 [00:24<00:00,  3.01s/it]


2025-10-15 10:36:09,312 - src.models.qwen_model - INFO - ✅ Qwen 모델 로딩 완료
Agent initialized
Development mode: False
Model info: {'model_name': 'Qwen/Qwen3-14B', 'device': 'cuda', 'dtype': 'float16', 'max_tokens': 512, 'temperature': 0.1, 'parameters': '14.8B', 'context_length': '32K (extendable to 128K)'}


### 4. 추천 에이전트 초기화

In [5]:
# Run evaluation on all test cases
all_eval_results = []
k_values = [3, 5, 10]

print(f"🔍 {len(all_test_cases)}개 테스트 케이스 평가 시작...")
print(f"평가 k 값: {k_values}")
print("=" * 80)

for idx, test_case in enumerate(tqdm(all_test_cases, desc="Evaluating")):
    try:
        # Extract test case info
        dataset_id = test_case['input_dataset']['svc_id']
        category = test_case['category']
        
        print(f"\n[{idx+1}/{len(all_test_cases)}] 평가 중: {test_case['input_dataset']['title']}")
        print(f"  데이터셋 ID: {dataset_id}")
        print(f"  카테고리: {category}")
        
        # Get recommendations from agent
        recommendation_result = await agent.recommend(dataset_id)
        
        if 'error' in recommendation_result:
            print(f"  ❌ 오류: {recommendation_result['error']}")
            continue
        
        # Convert agent output to evaluation format
        paper_recommendations = recommendation_result.get('paper_recommendations', [])
        dataset_recommendations = recommendation_result.get('dataset_recommendations', [])
        
        # Combine predictions
        predictions = []
        for paper in paper_recommendations:
            predictions.append({
                'type': 'paper',
                'id': paper['id'],
                'rank': paper['rank'],
                'score': paper['score']
            })
        
        for dataset in dataset_recommendations:
            predictions.append({
                'type': 'dataset',
                'id': dataset['id'],
                'rank': dataset['rank'],
                'score': dataset['score']
            })
        
        # Ground truth from test case
        ground_truth = test_case['candidate_pool']
        
        # Evaluate
        eval_result = evaluate_recommendations(predictions, ground_truth, k_values)
        
        # Add metadata
        eval_result['test_case_index'] = idx
        eval_result['dataset_id'] = dataset_id
        eval_result['category'] = category
        eval_result['input_title'] = test_case['input_dataset']['title']
        eval_result['processing_time_ms'] = recommendation_result.get('processing_time_ms', 0)
        
        all_eval_results.append(eval_result)
        
        # Print summary
        print(f"  결과: nDCG@5={eval_result.get('overall_ndcg@5', 0):.4f}, "
              f"MRR@5={eval_result.get('overall_mrr@5', 0):.4f}, "
              f"Recall@5={eval_result.get('overall_recall@5', 0):.4f}")
        
    except Exception as e:
        print(f"  ❌ 평가 중 예외 발생: {e}")
        logging.error(f"테스트 케이스 {idx} 평가 실패: {e}", exc_info=True)
        continue

print("\n" + "=" * 80)
print(f"✅ 평가 완료: {len(all_eval_results)}/{len(all_test_cases)}개 케이스 성공")

Starting evaluation on 19 test cases...
Evaluating with k values: [3, 5, 10]


Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]


[1/19] Evaluating: 한반도 기상 관측 데이터 세트
  Dataset ID: c1f03ab868f32371ec97a650bcdf39ac
  Category: 국내
2025-10-15 10:36:48,544 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c1f03ab868f32371ec97a650bcdf39ac
2025-10-15 10:36:48,662 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/c1f03ab868f32371ec97a650bcdf39ac?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-15 10:36:48,664 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset c1f03ab868f32371ec97a650bcdf39ac
2025-10-15 10:36:48,666 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "31 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "ctlg_type": "02",
    "dataset_type": "01",
    "ctlg_type_pc": "dataset",
    "dataset_type_pc": "국내",
    "dataset_pub_dt_pc": "2025",
    "dataset_access_type_


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.57it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 52.27it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 89.15it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 42.03it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.47it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 55.21it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 89.56it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 88.52it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.60it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.14it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 88.65it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 72.36it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 89.07it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 56.44it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 88.69it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.94it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 91.18it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 60.

2025-10-15 10:36:54,191 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 14개 데이터셋 순위 결정 완료


...


Batches: 100%|██████████| 1/1 [00:00<00:00, 70.79it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 88.43it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.64it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 61.88it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.51it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 60.96it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.41it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.68it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.75it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 91.59it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.79it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 82.06it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.10it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 82.73it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.23it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 84.59it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 67.09it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 86.

2025-10-15 10:42:28,560 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 14개 데이터셋 순위 결정 완료
2025-10-15 10:42:28,561 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2


2025-10-15 10:42:28,562 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 10:42:28,563 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 10:42:28,563 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 3 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 3 most relevant items.
## Source Dataset:
Title: Police recorded crime trends in Victoria during the COVID-19 pandemic
Description: The Crime Statistics Agency (CSA) has released a research paper entitled Police-recorded crime trends in Victoria during the COVID-19 pandemic. The paper examines trends in criminal offences and family violence incidents to 30 June 2020, focusing on comparing the period from 1 April 2020 to 30 June 2020 with the same period in 2019.\\r\\nThis paper aims to examine whether and how the COVID-19 pandemic, associated behavio

Evaluating: 100%|██████████| 19/19 [05:57<00:00, 18.82s/it]

  Results: nDCG@5=0.0000, MRR@5=0.0000, Recall@5=0.0000

Evaluation completed: 19/19 cases evaluated successfully


### 5. 테스트 케이스 평가 실행

In [6]:
# Save detailed individual results
individual_results_path = os.path.join(eval_output_dir, 'individual_results.json')
with open(individual_results_path, 'w', encoding='utf-8') as f:
    json.dump(all_eval_results, f, indent=2, ensure_ascii=False)

print(f"✅ 개별 결과 저장 완료: {individual_results_path}")
print(f"총 결과: {len(all_eval_results)}개")

Individual results saved to: /home/infidea/backup-data/paper-reco-agent/data/evaluation_results/20251015103509/individual_results.json
Total results: 19


### 6. 개별 결과 저장

In [7]:
if all_eval_results:
    # Convert to DataFrame for easier analysis
    df_results = pd.DataFrame(all_eval_results)
    
    print("\n" + "=" * 80)
    print("📊 평균 평가 메트릭")
    print("=" * 80)
    
    # Calculate metrics for each k
    metrics_summary = {}
    
    for k in k_values:
        print(f"\nk={k} 결과")
        print("-" * 80)
        
        # Overall metrics
        print(f"  전체 (Overall):")
        overall_metrics = {}
        for metric in ['ndcg', 'mrr', 'recall', 'precision']:
            col_name = f'overall_{metric}@{k}'
            if col_name in df_results.columns:
                mean_val = df_results[col_name].mean()
                std_val = df_results[col_name].std()
                overall_metrics[metric] = {'mean': mean_val, 'std': std_val}
                print(f"    - {metric.upper()}@{k}: {mean_val:.4f} ± {std_val:.4f}")
        
        # Paper metrics
        print(f"\n  논문 (Papers):")
        paper_metrics = {}
        for metric in ['ndcg', 'mrr', 'recall', 'precision']:
            col_name = f'paper_{metric}@{k}'
            if col_name in df_results.columns:
                mean_val = df_results[col_name].mean()
                std_val = df_results[col_name].std()
                paper_metrics[metric] = {'mean': mean_val, 'std': std_val}
                print(f"    - {metric.upper()}@{k}: {mean_val:.4f} ± {std_val:.4f}")
        
        # Dataset metrics
        print(f"\n  데이터셋 (Datasets):")
        dataset_metrics = {}
        for metric in ['ndcg', 'mrr', 'recall', 'precision']:
            col_name = f'dataset_{metric}@{k}'
            if col_name in df_results.columns:
                mean_val = df_results[col_name].mean()
                std_val = df_results[col_name].std()
                dataset_metrics[metric] = {'mean': mean_val, 'std': std_val}
                print(f"    - {metric.upper()}@{k}: {mean_val:.4f} ± {std_val:.4f}")
        
        metrics_summary[f'k{k}'] = {
            'overall': overall_metrics,
            'papers': paper_metrics,
            'datasets': dataset_metrics
        }
    
    # Save metrics summary
    metrics_summary_path = os.path.join(eval_output_dir, 'metrics_summary.json')
    with open(metrics_summary_path, 'w', encoding='utf-8') as f:
        json.dump(metrics_summary, f, indent=2, ensure_ascii=False)
    
    print(f"\n\n✅ 메트릭 요약 저장 완료: {metrics_summary_path}")
    
    # Statistics
    print("\n" + "=" * 80)
    print("📈 통계")
    print("=" * 80)
    print(f"  평균 추천 논문 수: {df_results['num_predicted_papers'].mean():.2f}개")
    print(f"  평균 추천 데이터셋 수: {df_results['num_predicted_datasets'].mean():.2f}개")
    print(f"  평균 관련 논문 수: {df_results['num_relevant_papers'].mean():.2f}개")
    print(f"  평균 관련 데이터셋 수: {df_results['num_relevant_datasets'].mean():.2f}개")
    print(f"  평균 처리 시간: {df_results['processing_time_ms'].mean():.0f} ms")
    
else:
    print("⚠️  분석할 평가 결과가 없습니다")


Average Evaluation Metrics

k=3 Results
--------------------------------------------------------------------------------
  Overall:
    - NDCG@3: 0.0000 ± 0.0000
    - MRR@3: 0.0000 ± 0.0000
    - RECALL@3: 0.0000 ± 0.0000
    - PRECISION@3: 0.0000 ± 0.0000

  Papers:
    - NDCG@3: 0.0000 ± 0.0000
    - MRR@3: 0.0000 ± 0.0000
    - RECALL@3: 0.0000 ± 0.0000
    - PRECISION@3: 0.0000 ± 0.0000

  Datasets:
    - NDCG@3: 0.4211 ± 0.5073
    - MRR@3: 0.4211 ± 0.5073
    - RECALL@3: 0.1155 ± 0.1760
    - PRECISION@3: 0.1754 ± 0.2322

k=5 Results
--------------------------------------------------------------------------------
  Overall:
    - NDCG@5: 0.1874 ± 0.2264
    - MRR@5: 0.1053 ± 0.1268
    - RECALL@5: 0.0499 ± 0.0734
    - PRECISION@5: 0.1053 ± 0.1393

  Papers:
    - NDCG@5: 0.0000 ± 0.0000
    - MRR@5: 0.0000 ± 0.0000
    - RECALL@5: 0.0000 ± 0.0000
    - PRECISION@5: 0.0000 ± 0.0000

  Datasets:
    - NDCG@5: 0.4211 ± 0.5073
    - MRR@5: 0.4211 ± 0.5073
    - RECALL@5: 0.1155 ± 

### 7. 타입별 메트릭 계산 및 저장

In [ ]:
if all_eval_results:
    print("\n" + "=" * 80)
    print("📊 카테고리별 메트릭 (k=5)")
    print("=" * 80)
    
    category_metrics = {}
    
    for category in df_results['category'].unique():
        category_data = df_results[df_results['category'] == category]
        
        print(f"\n{category} (n={len(category_data)}):")
        print("-" * 80)
        
        cat_metrics = {}
        for metric_type in ['overall', 'paper', 'dataset']:
            print(f"  {metric_type.capitalize()}:")
            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@5'
                if col_name in category_data.columns:
                    mean_val = category_data[col_name].mean()
                    type_metrics[metric] = mean_val
                    print(f"    - {metric.upper()}@5: {mean_val:.4f}")
            cat_metrics[metric_type] = type_metrics
        
        category_metrics[category] = cat_metrics
    
    # Save category metrics
    category_metrics_path = os.path.join(eval_output_dir, 'category_metrics.json')
    with open(category_metrics_path, 'w', encoding='utf-8') as f:
        json.dump(category_metrics, f, indent=2, ensure_ascii=False)
    
    print(f"\n\n✅ 카테고리 메트릭 저장 완료: {category_metrics_path}")
else:
    print("⚠️  분석할 평가 결과가 없습니다")

### 8. 카테고리별 분석

In [ ]:
if all_eval_results:
    # Export to CSV for easier analysis in spreadsheet tools
    csv_path = os.path.join(eval_output_dir, 'results.csv')
    df_results.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV 내보내기 완료: {csv_path}")
    print(f"컬럼: {list(df_results.columns)}")
    print(f"\nDataFrame 크기: {df_results.shape}")
    print(f"\n첫 몇 행:")
    print(df_results.head())
else:
    print("⚠️  내보낼 결과가 없습니다")

### 9. CSV로 결과 내보내기

In [ ]:
if all_eval_results:
    # Generate text summary report
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append("평가 요약 리포트")
    report_lines.append("=" * 80)
    report_lines.append(f"")
    report_lines.append(f"타임스탬프: {timestamp}")
    report_lines.append(f"모델: {settings.MODEL_NAME}")
    report_lines.append(f"임베딩 모델: {settings.EMBEDDING_MODEL}")
    report_lines.append(f"테스트 케이스: {len(all_eval_results)}/{len(all_test_cases)}")
    report_lines.append(f"")
    report_lines.append(f"모델 설정:")
    report_lines.append(f"  - Temperature: {settings.TEMPERATURE}")
    report_lines.append(f"  - Max Tokens: {settings.MAX_TOKENS}")
    report_lines.append(f"  - Paper Hybrid (alpha/beta): {settings.PAPER_HYBRID_ALPHA}/{settings.PAPER_HYBRID_BETA}")
    report_lines.append(f"  - Dataset Hybrid (alpha/beta): {settings.DATASET_HYBRID_ALPHA}/{settings.DATASET_HYBRID_BETA}")
    report_lines.append(f"")
    report_lines.append("=" * 80)
    report_lines.append("전체 메트릭")
    report_lines.append("=" * 80)
    
    for k in k_values:
        report_lines.append(f"")
        report_lines.append(f"k={k}:")
        report_lines.append("-" * 40)
        for metric_type in ['overall', 'paper', 'dataset']:
            report_lines.append(f"  {metric_type.capitalize()}:")
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    report_lines.append(f"    {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")
    
    report_lines.append(f"")
    report_lines.append("=" * 80)
    report_lines.append("카테고리별 분석 (k=5)")
    report_lines.append("=" * 80)
    
    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]
        report_lines.append(f"")
        report_lines.append(f"{category} (n={len(category_data)}):")
        report_lines.append("-" * 40)
        
        for metric_type in ['overall', 'paper', 'dataset']:
            col_name = f'{metric_type}_ndcg@5'
            if col_name in category_data.columns:
                mean_val = category_data[col_name].mean()
                report_lines.append(f"  {metric_type.capitalize()} nDCG@5: {mean_val:.4f}")
    
    report_lines.append(f"")
    report_lines.append("=" * 80)
    report_lines.append("리포트 끝")
    report_lines.append("=" * 80)
    
    # Save report
    report_path = os.path.join(eval_output_dir, 'summary_report.txt')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_lines))
    
    print('\n'.join(report_lines))
    print(f"\n\n✅ 요약 리포트 저장 완료: {report_path}")
else:
    print("⚠️  리포트를 생성할 결과가 없습니다")

## 평가 완료

모든 결과가 타임스탬프 디렉토리에 저장되었습니다:
- `config.json`: 평가 설정 및 모델 옵션
- `individual_results.json`: 각 테스트 케이스의 상세 결과
- `metrics_summary.json`: k별 평균 메트릭
- `category_metrics.json`: 카테고리별 메트릭 분석
- `results.csv`: 스프레드시트 분석용 테이블 데이터
- `summary_report.txt`: 사람이 읽기 쉬운 요약 리포트